In [1]:
from qiskit.circuit.library import QFT, QuantumVolume
from disqco.circuits.cp_fraction import cp_fraction, cz_fraction
from qiskit import transpile
from QASMBench.interface.qiskit import QASMBenchmark
from disqco.graphs.quantum_network import QuantumNetwork, linear_coupling, grid_coupling, all_to_all, tree_network
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph


num_qubits = 32

# circuit = cp_fraction(num_qubits, num_qubits, fraction=0.5, seed=42)

circuit = QFT(num_qubits, do_swaps=False)
# 
# circuit = QuantumVolume(num_qubits, depth=num_qubits)
# Transpile the circuit to the basis gates
basis_gates = ['u', 'cp']
circuit = transpile(circuit, basis_gates=basis_gates)

num_partitions = 8  # Define the number of partitions or QPUs you will distribute over
depth = circuit.depth()

qpu_size = num_qubits // num_partitions + 1 # Define the number of qubits per QPU. For simplicity, we divide qubits evenly and add one additional space to each for teleportation (an extra space is needed for the teleportation qubit so the algorithmr requires it!)
qpu_sizes = [qpu_size] * num_partitions # Store the number of qubits per QPU in a list

# Create a quantum network with the specified number of qubits and partitions

# If we do not specificy the connectivity, we have all-to-all by default.
# coupling = grid_coupling(num_partitions)
coupling = linear_coupling(num_partitions)  # Linear coupling for a linear topology
# coupling = all_to_all(num_partitions)  # Fully connected network
quantum_network = QuantumNetwork(qpu_sizes, coupling)

graph = QuantumCircuitHyperGraph(circuit, group_gates=False)





print(f'Number of qubits in circuit {circuit.num_qubits}')
print(f'Circuit depth: {circuit.depth()}')

Number of qubits in circuit 32
Circuit depth: 63


In [2]:
from disqco.parti.FM.fiduccia import *

FM_partitioner = FiducciaMattheyses(circuit=circuit,
                                    hypergraph=graph,
                                    network=quantum_network,
                                    graph=graph
                                    )

In [3]:
# results = FM_partitioner.partition()

# graph = FM_partitioner.hypergraph

# print(f'Best cost: {results['best_cost']}')


# if len(graph.nodes) < 500:
#     %load_ext jupyter_tikz
#     graph.draw(assignment=results['best_assignment'], qpu_info=qpu_sizes)





In [4]:
# from disqco.parti.genetic.genetic_algorithm_beta import *

# GA_partitioner = GeneticPartitioner(circuit=circuit, network=quantum_network, group_gates=True, log=True, multi_process=True)

# results = GA_partitioner.partition()
# print(f'Best cost: {results['best_cost']}')

In [5]:
from disqco.graphs.coarsening.coarsener import HypergraphCoarsener

coarsener = HypergraphCoarsener()

coarseneing_method = coarsener.coarsen_recursive_batches_mapped

results = FM_partitioner.partition(coarsener=coarseneing_method)

In [6]:
print(f'Best cost: {results['best_cost']}')

Best cost: 616


In [ ]:
results = FM_partitioner.net_coarsened_partition(hypergraph_coarsener=coarsener.coarsen_recursive_subgraph_batch)

print(f'Net coarsened partitioning cost: {results["best_cost"]}')

TypeError: run_full_net_coarsened_FM() got an unexpected keyword argument 'coarsener'

In [ ]:
# results = GA_partitioner.multilevel_partition(coarsener=coarseneing_method)

In [ ]:
# from disqco.graphs.quantum_network import linear_coupling

# coupling = linear_coupling(num_partitions)

# network = QuantumNetwork(qpu_sizes, qpu_connectivity=coupling)
# FM_partitioner = FiducciaMattheyses(circuit=circuit,
#                                     network=network)


# results = FM_partitioner.partition(log=True)

In [ ]:
# results = FM_partitioner.multilevel_partition()

In [ ]:
from disqco.parti.FM.fiduccia_ext import *
from disqco.parti.FM.FM_methods_ext import *

best_assignment = results["best_assignment"]
initial_qubit_assignment = best_assignment.transpose()

print(initial_qubit_assignment)

FM_partitioner = FiducciaMattheysesExt(circuit=circuit, network=quantum_network,initial_qubit_assignment=initial_qubit_assignment)

results = FM_partitioner.partition(passes=10, stochastic=False, log=True)



[[0 0 0 ... 3 3 3]
 [0 0 0 ... 3 3 3]
 [0 0 0 ... 3 3 3]
 ...
 [6 6 6 ... 7 7 7]
 [6 6 6 ... 7 7 7]
 [6 6 6 ... 6 6 6]]
Initial cost: 205.0
Pass  0: cost = 181.0
Pass  1: cost = 177.0
Pass  2: cost = 175.0
Pass  3: cost = 174.0
Pass  4: cost = 174.0


KeyboardInterrupt: 

In [ ]:
graph = FM_partitioner.hypergraph
%load_ext jupyter_tikz
graph.draw(qubit_assignment=results['best_qubit_assignment'], gate_assignment=results['best_gate_assignment'],  qpu_info=qpu_sizes,show_labels=False)

s/tikzlibrarytopaths.code.tex))))
(/usr/local/texlive/2024/texmf-dist/tex/generic/pgf/frontendlayer/tikz/librarie
s/tikzlibrarycalc.code.tex)
(/usr/local/texlive/2024/texmf-dist/tex/generic/pgf/frontendlayer/tikz/librarie
s/tikzlibrarybackgrounds.code.tex)
(/usr/local/texlive/2024/texmf-dist/tex/latex/l3backend/l3backend-pdftex.def)
No file 1e5117e9de06b756df00988e51fb6c94.aux.
(/usr/local/texlive/2024/texmf-dist/tex/context/base/mkii/supp-pdf.mkii
[Loading MPS to PDF converter (version 2006.09.02).]
) (/usr/local/texlive/2024/texmf-dist/tex/latex/epstopdf-pkg/epstopdf-base.sty
(/usr/local/texlive/2024/texmf-dist/tex/latex/latexconfig/epstopdf-sys.cfg))
Runaway definition?
->
! TeX capacity exceeded, sorry [main memory size=5000000].
\pgf@sys@bp ...rrentprotocol {\the \pgfutil@toks@ 
                                                  }}
l.6027 ...greyStyle] (n_7_58) at (3.005,6.562) {};
                                                  
!  ==> Fatal error occurred, no output PDF file pr